In [ ]:
# scikit-hep
from coffea.nanoevents import NanoAODSchema
from coffea.dataset_tools import (
    apply_to_fileset,
    max_chunks,
    preprocess,
)
import dask

# this analysis
import utils
from utils.processor import ZprimeAnalysis

/Users/moaly/miniforge3/envs/zprime_diff_coffea/lib/python3.11/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


### Load data from NTuples without materialising in memory

!! Install `xrootd` via `conda-forge`

In [2]:
N_FILES_MAX_PER_SAMPLE = 2
fileset = utils.input_files.construct_fileset(
    N_FILES_MAX_PER_SAMPLE,
)
print(fileset)

{'signal__nominal': {'files': {'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/ZprimeToTT_M2000_W20_TuneCP2_PSweights_13TeV-madgraph-pythiaMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/280000/45046F71-EB72-974D-BF4A-56A9C1B44524.root': 'Events', 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/ZprimeToTT_M2000_W20_TuneCP2_PSweights_13TeV-madgraph-pythiaMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/270000/22BAB5D2-9E3F-E440-AB30-AE6DBFDF6C83.root': 'Events'}, 'metadata': {'process': 'signal', 'variation': 'nominal', 'nevts': 523001, 'xsec': 1.0}}, 'ttbar_semilep__nominal': {'files': {'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/280000/0605BFCA-C65A-CA4C-8A6A-9148DC1B7B9A.root': 'Events', 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NAN

In [3]:

NanoAODSchema.warn_missing_crossrefs = False # silences warnings about branches we will not use here
#executor = processor.FuturesExecutor(workers=4)

dataset_runnable, dataset_updated = preprocess(
    fileset,
    align_clusters=False,
    step_size=100_000,
    files_per_batch=1,
    skip_bad_files=True,
    save_form=False,
)
to_compute = apply_to_fileset(
                ZprimeAnalysis(),
                max_chunks(dataset_runnable, 300),
                schemaclass=NanoAODSchema,
            )
(out,) = dask.compute(to_compute)

Processing signal with variation nominal
Fields: ['SV', 'GenPart', 'Flag', 'DeepMETResolutionTune', 'LHEReweightingWeight', 'L1simulation', 'GenJetAK8', 'LHEPdfWeight', 'fixedGridRhoFastjetCentral', 'LHEWeight', 'SoftActivityJetHT5', 'SoftActivityJetHT', 'HLTriggerFirstPath', 'SoftActivityJetNjets2', 'Tau', 'FsrPhoton', 'genWeight', 'GenVtx', 'RawMET', 'LowPtElectron', 'HLTriggerFinalPath', 'DeepMETResponseTune', 'boostedTau', 'GenMET', 'HLT', 'GenVisTau', 'SoftActivityJet', 'btagWeight', 'Pileup', 'fixedGridRhoFastjetCentralChargedPileUp', 'TkMET', 'SoftActivityJetNjets10', 'L1', 'SubGenJetAK8', 'HTXS', 'PSWeight', 'Electron', 'LHEScaleWeight', 'MET', 'GenDressedLepton', 'OtherPV', 'PV', 'Generator', 'Jet', 'SoftActivityJetNjets5', 'GenIsolatedPhoton', 'RawPuppiMET', 'fixedGridRhoFastjetCentralNeutral', 'luminosityBlock', 'PuppiMET', 'L1Reco', 'FatJet', 'CaloMET', 'GenJet', 'SoftActivityJetHT2', 'Muon', 'CorrT1METJet', 'genTtbarId', 'SoftActivityJetHT10', 'SubJet', 'LHEPart', 'IsoTrac

In [5]:
print(out.keys())
print(out["data__nominal"]["hist_dict"]["Zprime_channel"]) # nominal histogram missing

dict_keys(['signal__nominal', 'ttbar_semilep__nominal', 'ttbar_had__nominal', 'ttbar_lep__nominal', 'wjets__nominal', 'data__nominal'])
Hist(
  Regular(50, 0, 3000, name='observable', label='$m_{t\\bar{t}}$ [GeV]'),
  StrCategory([], growth=True, name='process', label='Process'),
  StrCategory([], growth=True, name='variation', label='Systematic variation'),
  storage=Weight()) # Sum: WeightedSum(value=0, variance=0)
